In [1]:
#importing the libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error, r2_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import spacy
from nltk.tokenize import word_tokenize
from keras_preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize




nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nlp = spacy.load("en_core_web_sm")






[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
test_csv = pd.read_csv("WednesdayTest.csv")

In [6]:
test_csv.head()

,SampleID,Discussion
0,1,Possibly he is a misspelling of the hero of Ay...
1,2,Like anal fisting teens?
2,3,Look into these sites:\nhttp://www.ukparks.com...
3,4,"NOVEMBER 03, 2004 (IDG NEWS SERVICE) - An anon..."
4,5,Of course they take a SMALL commissioned rate ...


In [8]:
from tqdm import tqdm
tqdm.pandas()
#####################################   PREPROCESSING    #####################################

########### 1. FILLING NULLS

print("\nMissing Values in Test Dataset:")
print(test_csv.isnull().sum())

test_csv = test_csv.fillna('')
print("\nMissing Values in Test Dataset After Filling:")
print(test_csv.isnull().sum())


########### 4. REMOVE NOISE FROM TEXT

def normalize_repeated_chars(word):
    repeat_regexp = re.compile(r'(.)\1{2,}')  # Match characters repeated 3 or more times
    word = repeat_regexp.sub(r'\1', word)  # Replace them with a single occurrence
    return word

def preprocess_text(text):
    # (handles NaN or other types)
    if not isinstance(text, str):
        return ""
    # Dictionary of contractions
    contraction_dict = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because",
                        "could've": "could have",
                        "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not",
                        "hadn't": "had not",
                        "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
                        "he's": "he is",
                        "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                        "I'd": "I would",
                        "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                        "I've": "I have", "i'd": "i would",
                        "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am",
                        "i've": "i have",
                        "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
                        "it'll've": "it will have",
                        "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                        "might've": "might have",
                        "mightn't": "might not", "mightn't've": "might not have", "must've": "must have",
                        "mustn't": "must not",
                        "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                        "o'clock": "of the clock",
                        "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                        "sha'n't": "shall not",
                        "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                        "she'll": "she will",
                        "she'll've": "she will have", "she's": "she is", "should've": "should have",
                        "shouldn't": "should not",
                        "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "this's": "this is",
                        "that'd": "that would",
                        "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                        "there'd've": "there would have",
                        "there's": "there is", "here's": "here is", "they'd": "they would",
                        "they'd've": "they would have",
                        "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                        "they've": "they have",
                        "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
                        "we'll": "we will",
                        "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                        "what'll": "what will",
                        "what'll've": "what will have", "what're": "what are", "what's": "what is",
                        "what've": "what have",
                        "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                        "where've": "where have",
                        "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                        "why's": "why is",
                        "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                        "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                        "y'all": "you all",
                        "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are",
                        "y'all've": "you all have",
                        "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                        "you'll've": "you will have",
                        "you're": "you are", "you've": "you have"}

    def _get_contractions(contraction_dict):
        contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
        return contraction_dict, contraction_re

    def replace_contractions(text):
        contractions, contractions_re = _get_contractions(contraction_dict)

        def replace(match):
            return contractions[match.group(0)]

        return contractions_re.sub(replace, text)

    text = replace_contractions(text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    # Remove user mentions (e.g., @username)
    text = re.sub(r'@\w+', '', text)
    # Replace weird things like \n
    text = re.sub(r'\\n', ' ', text)
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Normalize repeated characters (e.g., "noooooooooooo" -> "no")
    words = text.split()
    words = [normalize_repeated_chars(word) for word in words]
    text = " ".join(words)

    emoticon_pattern = re.compile(r"[:;=8xX]-?[)D(P/\\\|]")
    text = emoticon_pattern.sub(r'', text)

    # Tokenize and lemmatize using SpaCy
    doc = nlp(text)  # Apply lemmatization after correction
    lemmatized_text = " ".join([token.lemma_ for token in doc if token.text not in stopwords.words('english')])

    return lemmatized_text.lower()

test_csv['Discussion'] = test_csv['Discussion'].progress_apply(preprocess_text)


########### 5. REMOVE STOPWORDS

stop = stopwords.words('english')
custom_stopwords = set(stop) - {'not', 'no', 'but'}  # Remove from stopwords 'not', 'no', 'but'
custom_stopwords = custom_stopwords.union({'use', 'one', 'go', 'think', 'make'})
test_csv['Discussion'] = test_csv['Discussion'].astype(str).apply(lambda z: " ".join(z for z in z.lower().split() if z not in custom_stopwords))




Missing Values in Test Dataset:
SampleID      0
Discussion    0
dtype: int64

Missing Values in Test Dataset After Filling:
SampleID      0
Discussion    0
dtype: int64


100%|██████████| 43/43 [00:01<00:00, 30.04it/s]


In [9]:
########## 6. TOKENIZE & PAD
from keras.preprocessing.sequence import pad_sequences  # Correct import

max_words = 10000
max_sequence_length = 480
tokenizer = Tokenizer(num_words=max_words)

path = r'models pickled\tokenizer.pkl'
# Load the pickled tokenizer
with open(path, "rb") as file:
    tokenizer = (pickle.load(file))

#tokenizer.fit_on_texts(test_csv) ############### heya dy elly et3amalaha pickle


test_seq = tokenizer.texts_to_sequences(test_csv['Discussion'])
test_pad = pad_sequences(test_seq, maxlen=max_sequence_length)



In [11]:
########### 8. LOAD MODEL (SIMPLE RNN)
from keras.models import load_model

simple_path = r'models pickled\rnn_model1_0.70.h5'
# Load the model
rnn_model1 = load_model(simple_path)
# Predict categories for the test data
rnn_model1_predictions = rnn_model1.predict(test_pad)



2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 343ms/step


In [27]:
rnn_model1_predictions

array([[0.0127262 , 0.01467933, 0.02964983, 0.16647701, 0.77646756],
       [0.07250869, 0.12786977, 0.2061121 , 0.1753914 , 0.418118  ],
       [0.00570137, 0.00270727, 0.01575111, 0.16671193, 0.8091282 ],
       ...,
       [0.15125938, 0.18022051, 0.25550494, 0.22327152, 0.18974374],
       [0.4217559 , 0.02322108, 0.03769602, 0.40575993, 0.11156708],
       [0.11939061, 0.10822439, 0.53914946, 0.18192254, 0.05131301]],
      dtype=float32)

In [12]:
predicted_classes = rnn_model1_predictions.argmax(axis=1)

sample_ids = np.arange(1, len(predicted_classes) + 1)
    
output_df = pd.DataFrame({
    "SampleID": sample_ids,
    "Category": predicted_classes
})
    
output_df.to_csv('rnn1_test.csv', index=False)
print(output_df.head())

   SampleID  Category
0         1         2
1         2         2
2         3         4
3         4         4
4         5         3


In [13]:
rnn_path = r'models pickled\rnn_model2_0.723.h5'
# Load the model
rnn_model2 = load_model(rnn_path)
rnn_model2_predictions = rnn_model2.predict(test_pad)


predicted_classes = rnn_model2_predictions.argmax(axis=1)

sample_ids = np.arange(1, len(predicted_classes) + 1)
    
output_df = pd.DataFrame({
    "SampleID": sample_ids,
    "Category": predicted_classes
})
    
output_df.to_csv('rnn2_test.csv', index=False)
print(output_df.head())

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step
   SampleID  Category
0         1         4
1         2         2
2         3         4
3         4         4
4         5         3


In [14]:
# Load the model
cnn_path = r'models pickled\cnn_model_0.717.h5'
cnn_model = load_model(cnn_path)
cnn_model_predictions = cnn_model.predict(test_pad)

predicted_classes = cnn_model_predictions.argmax(axis=1)

sample_ids = np.arange(1, len(predicted_classes) + 1)
    
output_df = pd.DataFrame({
    "SampleID": sample_ids,
    "Category": predicted_classes
})
    
output_df.to_csv('cnn_test.csv', index=False)
print(output_df.head())

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000214E42C7380> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
   SampleID  Category
0         1         2
1         2         2
2         3         2
3         4         4
4         5         3


In [ ]:
def get_positional_encoding(seq_len, embed_dim):
    positional_encoding = np.zeros((seq_len, embed_dim))

    for pos in tqdm(range(seq_len), desc="Generating Positional Encoding"):
        for i in range(embed_dim):
            angle = pos / np.power(10000, (2 * (i // 2)) / np.float32(embed_dim))
            if i % 2 == 0:
                positional_encoding[pos, i] = np.sin(angle)  # Apply sine to even indices
            else:
                positional_encoding[pos, i] = np.cos(angle)  # Apply cosine to odd indices
    return tf.cast(positional_encoding, dtype=tf.float32)


In [ ]:
from tensorflow.keras.layers import MultiHeadAttention
transformer_path = r'models pickled\\transformer_model_0.71.h5'

transformer_model = load_model(
    transformer_path, 
    custom_objects={'get_positional_encoding': get_positional_encoding, 
                    'MultiHeadAttention': MultiHeadAttention}
)